In [74]:
from RDSDatabaseConnector import RDSDatabaseConnector
from Plotter import Plotter
from DataTransform import DataTransform
from DataFrameTransform import DataFrameTransform
from DataFrameInfo import DataFrameInfo
from functions import display_suggested_drops, column_drop
import pandas as pd
import numpy as np

from datetime import datetime

RDSD = RDSDatabaseConnector()
PL = Plotter()
DT = DataTransform()
DFT = DataFrameTransform()
DFI = DataFrameInfo()

print("BEGINNING OF PROGRAM\n")
print("ASSESSING")
datacsv_df = pd.read_csv('data.csv') #define dataframe
current_all_columns = list(datacsv_df.columns) #define list of all columns in dataframe

#columns to convert to the various formats:
list_of_to_categorical = ['term','grade','sub_grade', 'employment_length','home_ownership','verification_status','loan_status','payment_plan','purpose','delinq_2yrs','application_type']
list_of_to_boolean = ['policy_code']
list_of_to_float = []
list_of_to_int = []
date_columns = ['issue_date','earliest_credit_line','last_payment_date','next_payment_date','last_credit_pull_date']
acceptable_null_percentage: float = 80.0
print("ASSESSING COMPLETE")

def initial_general_cleanup(dataframe: pd.core.frame.DataFrame, acceptable_null_percentage):
    """Perform cleanup, return dataframe with converted dtypes, no null values.
    Specific cleanup processes to
    be listed here in detail:
    
    Perform Conversions - dates, categorical, boolean, float, int.
    Drop Null columns under X% full of rows.
    Impute columns with data.
    
    args:
    dataframe (type: pd.core.frame.DataFrame)
    acceptable_null_percentage (type: float)
    
    """
    #Column Conversions:
    dataframe = DT.convert_dates_to_proper_format(dataframe, date_columns, format = '%b-%Y') # %b is Jan/Feb/Mar etc
    dataframe = DT.to_categorical(dataframe, list_of_to_categorical)
    dataframe = DT.to_boolean(dataframe, list_of_to_boolean)
    dataframe = DT.to_float(dataframe, list_of_to_float)
    dataframe = DT.to_int(dataframe, list_of_to_int)
    
    #Drop Null columns under X%:
    acceptable_null_percentage = display_suggested_drops(dataframe, acceptable_null_percentage)
    dataframe, dropped_columns= column_drop(dataframe, acceptable_null_percentage)
    
    #Impute the rest of the columns:
    dataframe = DFT.impute(dataframe)
    
    return dataframe, dropped_columns

#Perform cleanup, return dataframe with converted dtypes, no null values.
print("\n")
print("Perform cleanup, return dataframe with converted dtypes, no null values.")
clean_dataframe, dropped_columns = initial_general_cleanup(datacsv_df, acceptable_null_percentage)
print("CLEANUP COMPLETE")

#remove dropped columns from current_all_columns variable.
print("\n")
print("Modify current_all_columns to reflect the removal of the previously stated columns from the dataframe")
for i in dropped_columns:
    current_all_columns.remove(i)
print("MODIFICATIONS COMPLETE")
### UP TO HERE YIELDS A DATAFRAME CALLED clean_dataframe.

###OPTIONAL CODE FOR SKEW CORRECTION BEGIN
'''#THE PART WHERE SKEW MAY BE IMPORTANT
#Check for skew in the dataframe
#Visualise skew in the dataframe
#Apply skew correction for each column if (on a per column basis) it is necessary
#PL.skew_correction(clean_dataframe)

#dataframe = apply_skew_correction(dataframe)
'''
###OPTIONAL CODE FOR SKEW CORRECTION END

def remove_columns_for_analysis(dataframe, columns):
    def remove_columns(dataframe, columns):
        clean_dataframe, symbols_removed = DT.excess_symbol_removal(dataframe, columns)
        return clean_dataframe, symbols_removed

    symbols_to_be_removed_from_dataframe = [columns]
    dataframe, symbols_removed = remove_columns(dataframe, symbols_to_be_removed_from_dataframe)
    for i in symbols_removed:
        dropped_columns.append(i)#append symbols to be removed to dropped_columns
    return dataframe

# Remove excess symbols
symbols = ['id', 'member_id', 'term', 'int_rate', 'grade', 'sub_grade', 'employment_length',
           'verification_status', 'issue_date', 'payment_plan', 'purpose', 'dti', 'delinq_2yrs', 'home_ownership',
           'earliest_credit_line', 'inq_last_6mths', 'open_accounts', 'total_accounts', 'last_payment_date',
           'last_credit_pull_date', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'annual_inc']

dataframe_for_MS4_T2_analysis = remove_columns_for_analysis(clean_dataframe, symbols)
### UP TO HERE YIELDS A DATAFRAME CALLED dataframe_for_MS4_T1_analysis.
print("\n")

"""Milestone 4 Task 2

The company wants to check what percentage of loans have been a loss to the company:

Loans marked as Charged Off in the loan_status column represent a loss to the company.


Calculate the percentage of charged off loans historically and the total amount that 
was paid towards these loans before being charged off.
"""
def calculate_the_percentage_of_charged_off_loans_historically(dataframe: pd.core.frame.DataFrame)-> pd.core.frame.DataFrame:
    variable_to_display_the_number_of_chargedoff_in_dataframe = []
    for i in dataframe['loan_status']:
        if i == 'Charged Off':
            variable_to_display_the_number_of_chargedoff_in_dataframe.append(i)
    print("Number of charged off: \n", len(variable_to_display_the_number_of_chargedoff_in_dataframe))
    print("Percentage of charged off loans in dataframe: ", (len(variable_to_display_the_number_of_chargedoff_in_dataframe)/len(dataframe_for_MS4_T2_analysis))*100, "%")

calculate_the_percentage_of_charged_off_loans_historically(dataframe_for_MS4_T2_analysis)

#the total amount that was paid towards these loans before being charged off.
'''total_payment'+'total_rec_int'''
#- **total_payment**: Payments received to date for total amount funded
#- **total_rec_int**: Interest received to date
#- **total_rec_late_fee**: Late fees received to date

def create_new_column(dataframe: pd.core.frame.DataFrame, new_column: str)-> pd.core.frame.DataFrame:
    dataframe[f'{new_column}'] = np.nan
    return dataframe
dataframe_for_MS4_T2_analysis_with_new_column = create_new_column(dataframe_for_MS4_T2_analysis, new_column = 'tot_amount_paid_before_chgd_off')
#print(dataframe_for_MS4_T2_analysis_with_new_column)

def check_if_charged_off(dataframe: pd.core.frame.DataFrame)-> pd.core.frame.DataFrame:
    for i in dataframe['loan_status']:
        if i == 'Charged Off':
            pass #call function to apply <values> to column tot_amount_paid_before_chgd_off
    return dataframe

def apply_values_to_column():
    pass
#(dataframe.loc['total_payment'] + dataframe.loc['total_rec_int'])    
#dataframe_for_MS4_T2_analysis_with_new_column_mod = 
#check_if_charged_off(dataframe_for_MS4_T2_analysis_with_new_column)
#print(dataframe_for_MS4_T2_analysis_with_new_column_mod['tot_amount_paid_before_chgd_off'])

#def tot_amount_paid_before_chgd_off(dataframe_for_MS4_T2_analysis: pd.core.frame.DataFrame)-> pd.core.frame.DataFrame:
#    #tot_amount_paid_before_chgd_off = []
#    for i in dataframe_for_MS4_T2_analysis['loan_status']:
#        if dataframe_for_MS4_T2_analysis['loan_status'] == 'Charged Off':
#            print("the total amount that was paid towards these loans before being charged off: \n", dataframe_for_MS4_T2_analysis['total_payment']+dataframe_for_MS4_T2_analysis['total_rec_int'])
#            tot_amount_paid_before_chgd_off = 1#tot_amount_paid_before_chgd_off.append(dataframe_for_MS4_T2_analysis['total_payment']+dataframe_for_MS4_T2_analysis['total_rec_int'])
#    return None#tot_amount_paid_before_chgd_off







BEGINNING OF PROGRAM

ASSESSING
ASSESSING COMPLETE


Perform cleanup, return dataframe with converted dtypes, no null values.
Suggest dropping these columns: 
 ['mths_since_last_delinq', 'mths_since_last_record', 'next_payment_date', 'mths_since_last_major_derog']
Dropped columns: 
 ['mths_since_last_delinq', 'mths_since_last_record', 'next_payment_date', 'mths_since_last_major_derog']
CLEANUP COMPLETE


Modify current_all_columns to reflect the removal of the previously stated columns from the dataframe
MODIFICATIONS COMPLETE
These are the symbols that will be dropped:  [['id', 'member_id', 'term', 'int_rate', 'grade', 'sub_grade', 'employment_length', 'verification_status', 'issue_date', 'payment_plan', 'purpose', 'dti', 'delinq_2yrs', 'home_ownership', 'earliest_credit_line', 'inq_last_6mths', 'open_accounts', 'total_accounts', 'last_payment_date', 'last_credit_pull_date', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'annual_inc']]
These are the remaining symbols